# T1562 - Impair Defenses
Adversaries may maliciously modify components of a victim environment in order to hinder or disable defensive mechanisms. This not only involves impairing preventative defenses, such as firewalls and anti-virus, but also detection capabilities that defenders can use to audit activity and identify malicious behavior. This may also span both native defenses as well as supplemental capabilities installed by users and administrators.

Adversaries may also impair routine operations that contribute to defensive hygiene, such as blocking users from logging out of a computer or stopping it from being shut down. These restrictions can further enable malicious operations as well as the continued propagation of incidents.(Citation: Emotet shutdown)

Adversaries could also target event aggregation and analysis mechanisms, or otherwise disrupt these procedures by altering other system components.

## Atomic Tests

### Atomic Test #1 - Windows Disable LSA ProtectionThe following Atomic adds a registry entry to disable LSA Protection.

The LSA controls and manages user rights information, password hashes and other important bits of information in memory. Attacker tools, such as mimikatz, rely on accessing this content to scrape password hashes or clear-text passwords. Enabling LSA Protection configures Windows to control the information stored in memory in a more secure fashion - specifically, to prevent non-protected processes from accessing that data.
Upon successful execution, the registry will be modified and RunAsPPL will be set to 0, disabling Lsass protection.
https://learn.microsoft.com/en-us/windows-server/security/credentials-protection-and-management/configuring-additional-lsa-protection#how-to-disable-lsa-protection
https://blog.netwrix.com/2022/01/11/understanding-lsa-protection/
https://thedfirreport.com/2022/03/21/phosphorus-automates-initial-access-using-proxyshell/  
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKLM\SYSTEM\CurrentControlSet\Control\LSA /v RunAsPPL /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1562 -TestNumbers 1

#### Cleanup: 
```cmd
reg delete HKLM\SYSTEM\CurrentControlSet\Control\LSA /v RunAsPPL /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1562 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Disable journal logging via systemctl utilityThe atomic test disables the journal logging using built-in systemctl utility
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo systemctl stop systemd-journald #disables journal logging
```

In [ ]:
Invoke-AtomicTest T1562 -TestNumbers 2

#### Cleanup: 
```sh
sudo systemctl start systemd-journald #starts journal service
sudo systemctl enable systemd-journald #starts journal service automatically at boot time
```

In [ ]:
Invoke-AtomicTest T1562 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Disable journal logging via sed utilityThe atomic test disables the journal logging by searching and replacing the "Storage" parameter to "none" within the journald.conf file, thus any new journal entries will only be temporarily available in memory and not written to disk
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo sed -i 's/Storage=auto/Storage=none/' /etc/systemd/journald.conf
```

In [ ]:
Invoke-AtomicTest T1562 -TestNumbers 3

#### Cleanup: 
```sh
sudo sed -i 's/Storage=none/Storage=auto/' /etc/systemd/journald.conf #re-enables storage of journal data
sudo systemctl restart systemd-journald #restart the journal service
```

In [ ]:
Invoke-AtomicTest T1562 -TestNumbers 3 -Cleanup

## Detection
Monitor processes and command-line arguments to see if security tools or logging services are killed or stop running. Monitor Registry edits for modifications to services and startup programs that correspond to security tools.  Lack of log events may be suspicious.

Monitor environment variables and APIs that can be leveraged to disable security measures.


## Shield Active Defense
### Application Diversity 
 Present the adversary with a variety of installed applications and services. 

 Application diversity is presenting multiple software targets to the adversary. On a single target system, defenders can configure multiple different services or user software applications.  On a target network, defenders can present systems with a variety of operating systems, operating system versions, applications, and services.
#### Opportunity
There is an opportunity to study the adversary and collect first-hand observations about them and their tools.
#### Use Case
A defender can plant AV or monitoring tools which are easy for an adversary to remove. If an adversary removes these, they may be enticed to act more openly believing they have removed monitoring from the system.
#### Procedures
Use a mix of vulnerable and nonvulnerable software on a system to allow you to see what exploits the adversary leverages in their attacks.
Install Anti-virus or other end-point detection tools on systems to see if an adversary takes note of them and if so, how they react.
